## Example of metric calculation using moving object metrics.

The inputs needed here (the obsfile) can be calculated using the code demonstrated in the [ExampleMoObs](./ExampleMoObs.ipynb) notebook. 

Here, I'm working through how those observations can be read back into the code and metric values (such as discovery chances and then completeness) can be calculated. 

There are definitely some things to be improved upon; if the observation generation stopped at the point of generating interpolations, for example, and the moving object slicer read those interpolations back in, together with the the simdata, it could generate the actual observations on the fly (when using the camera footprint, this would add non-insignificant overhead, so it might actually be nice to be able to do either/both). 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from moSlicer import MoSlicer
import moMetrics as MoMetrics
from moSecondaryMetrics import MoCompletenessMetric, IntegrateOverHMetric
import moPlots as moPlots
import moMetricBundle as mmb

SyntaxError: invalid syntax (moMetricBundle.py, line 51)

In [ ]:
# Set up MoSlicer.  We use the moving object slicer to read the observation data from disk.
orbitfile = 'pha20141031.des'
obsfile = 'pha_withChip_allObs.txt'

In [ ]:
mos = MoSlicer(orbitfile, Hrange=np.arange(13, 25, 0.5))
mos.readObs(obsfile)
mos.allObs.tail(10)

In [ ]:
mos.orbits.tail()

In [ ]:
### side note.. illustrating detection probability curve (50% at m5)
# this probability curve is used if snrLimit is NOT set (otherwise it's just a cutoff at snrLimit)
metric = MoMetrics.BaseMoMetric()
# Check what 'visibility' looks like. 
appMag = np.arange(18, 27, 0.001)
magLim = 22.8
vis = metric._calcVis(appMag, magLim)
bins = np.arange(appMag.min(), appMag.max(), 0.1)
counts = np.zeros(len(bins))
for i, (b1, b2) in enumerate(zip(bins[:-1], bins[1:])):
    counts[i] = len(np.where((appMag >= b1) & (appMag < b2) & (vis==1))[0])
plt.plot(bins, counts)

In [ ]:
# Set up an example metric bundle.
metric = MoMetrics.NObsMetric()
slicer = mos
pandasConstraint = ''
mobundle = mmb.MoMetricBundle(metric, slicer, pandasConstraint)

In [ ]:
# Put this into a dictionary and pass to the moMetricBundleGroup. 
bdict = {'nobs':mobundle}
bg = mmb.MoMetricBundleGroup(bdict)

In [ ]:
bg.runAll()

In [ ]:
print mobundle.metricValues

In [ ]:
pl = moPlots.MetricVsH()
pl(mobundle.metricValues, mos, {'ylabel':'Nobs'})

In [ ]:
pl = moPlots.MetricVsOrbitPoints()
pl(mobundle.metricValues, mos, {})